In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path
import numpy as np
import random


from utils.data import make_trees_dataset
from utils.nnet import get_device

from utils.nnet import from_gpu
from utils.eval import compute_accuracy
from hebbcl.logger import LoggerFactory
from hebbcl.model import ScaledNet2Hidden, ScaledNet2Hidden2Ctx, Nnet
from hebbcl.trainer import Optimiser, train_on_trees
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

In [ ]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.cuda = False
args.device = get_device(args.cuda)[0]
# override defaults 
# args.n_features = None # todo need to change to flattened size of image
args.n_episodes = 200
args.training_schedule = "interleaved"
args.n_features = 1730
args.n_hidden = 100
args.lrate_sgd = 0.01
args.lrate_hebb = 0.03
args.n_layers = 1
args.ctx_twice = False
args.perform_hebb = False
args.ctx_scaling = 2
args.ctx_avg = False
args.centering = False

# create dataset 
dataset = make_trees_dataset(args)

# instantiate model and optimiser:
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)
logger = LoggerFactory.create(args, save_dir=save_dir)

# train model
train_on_trees(args, model, optimiser, dataset, logger)
print("done")


In [ ]:
logger = LoggerFactory.create(args, save_dir=save_dir)


In [ ]:
args.n_layers

In [ ]:
args.training_schedule